In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
data = pd.read_csv('../data/daily_category_sales.csv', index_col=0, header=[0,1])

data = data[~(data==0).any(axis=1)]

In [3]:
categories_enc =  {'Aquatic Roots and Tubers':'1', 'Cauliflower':'2' ,'Chili Peppers':'3', 'Edible Mushrooms':'4', 'Leafy Greens':'5', 'Solanaceous Vegetables':'6'}
data.columns = ['_'.join([col[0], categories_enc[col[1]]]) for col in data.columns]
data.reset_index(inplace=True)

for i in range(1,7):
    data[f'sales_sum_{i}'] = data[f'sales_profit_sum_{i}'] + data[f'cost_sum_{i}']
    data[f'avg_sale_price_{i}'] = data[f'sales_sum_{i}'] / data[f'quantity_sum_{i}']
data['sales_sum'] = data[[f'sales_sum_{i}' for i in range(1,7)]].sum(axis=1)


data.drop(columns=[f'sales_profit_sum_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'avg_profit_rate_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'cost_sum_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'sales_sum_{i}' for i in range(1,7)], inplace=True)

In [4]:
data.head()

,date,quantity_sum_1,quantity_sum_2,quantity_sum_3,quantity_sum_4,quantity_sum_5,quantity_sum_6,avg_sale_price_1,avg_sale_price_2,avg_sale_price_3,avg_sale_price_4,avg_sale_price_5,avg_sale_price_6,sales_sum
0,2020-07-01,4.850,47.076,76.715,35.365,205.402,35.374,14.491505,12.716331,9.906670,10.422791,7.321202,4.998530,3478.1176
1,2020-07-02,4.600,43.943,66.064,48.510,198.362,32.199,11.548000,12.425915,10.137206,12.490806,7.106526,5.344638,3456.5429
2,2020-07-03,9.572,42.076,64.253,42.442,191.779,35.896,8.012223,11.773172,10.437872,12.871566,7.333475,5.466988,3391.6701
3,2020-07-04,5.439,55.662,81.282,47.262,236.587,57.067,21.833646,12.615177,10.167277,12.244586,7.783183,5.737169,4394.8624
4,2020-07-05,4.019,55.474,98.496,73.213,223.899,61.816,10.000000,12.582147,10.317542,10.006745,7.026290,6.654638,4471.5748


### add dummy variables of season and day_of_week

In [4]:
data['date'] = pd.to_datetime(data['date'])
data['season'] = data['date'].dt.month%12 // 3 + 1 
data['day_of_week'] = data['date'].dt.dayofweek + 1 

In [5]:
season_dummies = pd.get_dummies(data['season'])
season_dummies.columns = ['winter', 'spring', 'summer', 'fall']
season_dummies.drop(columns=['fall'], inplace=True)

dayofweek_dummies = pd.get_dummies(data['day_of_week'])
dayofweek_dummies.columns = ['_'.join(['DoW', str(i)]) for i in range(1,8)]
dayofweek_dummies.drop(columns=['DoW_7'], inplace=True)

In [6]:
data = pd.concat([data, season_dummies, dayofweek_dummies], axis=1)
data.drop(columns=['season', 'day_of_week'], inplace=True)
data.head()

,date,quantity_sum_1,quantity_sum_2,quantity_sum_3,quantity_sum_4,quantity_sum_5,quantity_sum_6,avg_sale_price_1,avg_sale_price_2,avg_sale_price_3,...,sales_sum,winter,spring,summer,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6
0,2020-07-01,4.850,47.076,76.715,35.365,205.402,35.374,14.491505,12.716331,9.906670,...,3478.1176,0,0,1,0,0,1,0,0,0
1,2020-07-02,4.600,43.943,66.064,48.510,198.362,32.199,11.548000,12.425915,10.137206,...,3456.5429,0,0,1,0,0,0,1,0,0
2,2020-07-03,9.572,42.076,64.253,42.442,191.779,35.896,8.012223,11.773172,10.437872,...,3391.6701,0,0,1,0,0,0,0,1,0
3,2020-07-04,5.439,55.662,81.282,47.262,236.587,57.067,21.833646,12.615177,10.167277,...,4394.8624,0,0,1,0,0,0,0,0,1
4,2020-07-05,4.019,55.474,98.496,73.213,223.899,61.816,10.000000,12.582147,10.317542,...,4471.5748,0,0,1,0,0,0,0,0,0


### Double log regression

In [14]:
dummies = data[['winter', 'spring', 'summer', 'DoW_1', 'DoW_2', 'DoW_3', 'DoW_4', 'DoW_5', 'DoW_6']]
for i in range(1,7):
    print(f'\nformula {i}\n')
    category_name = list(categories_enc.keys())[i-1]
    category_name = category_name.replace(' ', '_')

    y_data = data[f'quantity_sum_{i}'].copy()
    log_y = y_data.apply(lambda x: np.log(x))
    X_data = data[['avg_sale_price_1', 'avg_sale_price_2', 'avg_sale_price_3', 'avg_sale_price_4', 'avg_sale_price_5', 'avg_sale_price_6', 'sales_sum']].copy()
    log_X = X_data.apply(lambda x: np.log(x))
    
    logX_d = pd.concat([log_X, dummies], axis=1)

    # first regression
    model = sm.OLS(log_y, logX_d)
    result1 = model.fit()
    summary1 = result1.summary()
    print(f'regression coefficient for quantity_sum of {category_name}')
    print(summary1.tables[1])

    coef = pd.DataFrame(summary1.tables[1].data[1:], columns=summary1.tables[1].data[0])
    coef.index = coef['']
    coef.drop(columns=[''], inplace=True)
    coef = coef.astype(float)

    # delete in sufficient variables from X
    insufficients_variables = coef[coef['P>|t|'] >= 0.05].index.tolist()
    print(f'\ninsufficient variables: {insufficients_variables}\n')
    logX_d.drop(columns=insufficients_variables, inplace=True)

    # re-fit regression model
    adjusted_model = sm.OLS(log_y, logX_d)
    result2 = adjusted_model.fit()
    summary2 = result2.summary()
    print(f'adjusted regression coefficient for quantity_sum of {category_name}')
    print(summary2.tables[1])

    # save regression coefficients
    final_coef = pd.DataFrame(summary2.tables[1].data[1:], columns=summary2.tables[1].data[0])
    final_coef.index = final_coef['']
    final_coef.drop(columns=[''], inplace=True)
    final_coef = final_coef.astype(float)

    final_coef.to_csv(f'../results/reg_coef_{category_name}.csv')


formula 1

regression coefficient for quantity_sum of Aquatic_Roots_and_Tubers
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
avg_sale_price_1    -1.1486      0.061    -18.835      0.000      -1.268      -1.029
avg_sale_price_2    -0.2501      0.069     -3.629      0.000      -0.385      -0.115
avg_sale_price_3     0.0273      0.049      0.553      0.580      -0.069       0.124
avg_sale_price_4    -0.5430      0.064     -8.439      0.000      -0.669      -0.417
avg_sale_price_5    -0.1647      0.081     -2.029      0.043      -0.324      -0.005
avg_sale_price_6     0.3884      0.075      5.166      0.000       0.241       0.536
sales_sum            0.9024      0.028     32.587      0.000       0.848       0.957
winter              -0.0060      0.054     -0.111      0.912      -0.111       0.100
spring              -0.6160      0.052    -11.842      0.000      -0.7